In [98]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import urllib.parse as p
import re
import os
import pickle

SCOPES = ["https://www.googleapis.com/auth/youtube.force-ssl"]

In [56]:
r = requests.get("https://pitchfork.com/features/lists-and-guides/best-songs-2021/")

soup = bs(r.content)

print(soup.prettify())

<!DOCTYPE html>
<html data-reactroot="" lang="en-US">
 <head>
  <title>
   The 100 Best Songs of 2021 | Pitchfork
  </title>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="no" name="msapplication-tap-highlight"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="no-referrer-when-downgrade" name="referrer"/>
  <meta content="Condé Nast" name="author"/>
  <meta content="Copyright (c) Condé Nast 2022" name="copyright"/>
  <meta content="The tracks that defined an unsettling year, featuring Japanese Breakfast, Caroline Polachek, Megan Thee Stallion, Low, and more" name="description"/>
  <meta content="61a66e4239ed78ff9e45e0b8" name="id"/>
  <meta content="best of 2021" name="keywords"/>
  <meta content="best of 2021" name="news_keywords"/>
  <meta content="index, follow, max-image-preview:large" name="robots"/>
  <meta content="article" name="content-type"/>
  <meta content="61a66e4239ed78ff9e45e0b8

In [81]:
chart = soup.find(class_ = "ArticlePageChunks-lgQXyi iPDsWW")
titles = chart.find_all("h2")
for index,title in enumerate(titles):
    titles[index] = str(titles[index]).replace("<h2>","").replace("</h2>","")


In [82]:
#Titles of each paragraph in the website
titles

['Halsey: “I am not a woman, I’m a god”',
 'Smerz: “Believer”',
 'Mandy, Indiana: “Bottle Episode”',
 'beabadoobee: “Last Day on Earth”',
 'Kay Flock: “Being Honest”',
 'Doss: “Look”',
 'Doja Cat: “Get Into It (Yuh)”',
 'Mavi: “Time Travel”',
 'SPELLLING: “Little Deer”\xa0',
 'Snoh Aalegra: “We Don’t Have to Talk About It”',
 'The Armed: “All Futures”',
 'Danny L Harle: “Boing Beat”',
 'Dijon: “Many Times”',
 'Joy Orbison: “better” [ft. Léa Sen]',
 'Mariah the Scientist: “2 You”',
 'City Girls: “Twerkulator”',
 'Ethel Cain: “Michelle Pfeiffer” [ft. lil aaron]',
 'Leo Bhanji: “Damaged”',
 'Shannon Lay: “Rare to Wake”',
 'Yeat: “Gët Busy”',
 'Rostam: “4Runner”',
 'Water From Your Eyes: ““Quotations””',
 'Dean Blunt: “the rot”',
 'Iceage: “Shelter Song”',
 'Foodman: “Parking Area”',
 'Bad Boy Chiller Crew: “Don’t You Worry About Me”',
 'Helado Negro: “Gemini and Leo”',
 'Indigo De Souza: “Hold U”',
 'Bruiser Wolf: “Dope Game $tupid”',
 'Lucy Dacus: “VBS”',
 'Jayda G: “All I Need”',
 'Norm

In [84]:
#Extracting artists names and song names from titles
artists = []
songs = []
for index, title in enumerate(titles):
    artists.append(titles[index].split(":",2)[0].lstrip())
    songs.append(titles[index].split(":",2)[1].lstrip())
print(artists)
print()
print(songs)


['Halsey', 'Smerz', 'Mandy, Indiana', 'beabadoobee', 'Kay Flock', 'Doss', 'Doja Cat', 'Mavi', 'SPELLLING', 'Snoh Aalegra', 'The Armed', 'Danny L Harle', 'Dijon', 'Joy Orbison', 'Mariah the Scientist', 'City Girls', 'Ethel Cain', 'Leo Bhanji', 'Shannon Lay', 'Yeat', 'Rostam', 'Water From Your Eyes', 'Dean Blunt', 'Iceage', 'Foodman', 'Bad Boy Chiller Crew', 'Helado Negro', 'Indigo De Souza', 'Bruiser Wolf', 'Lucy Dacus', 'Jayda G', 'Normani', 'Turnstile', 'Yu Su', 'Dawn Richard', 'Armand Hammer and the Alchemist', 'serpentwithfeet', 'Backxwash', 'Lil Nas X', 'Cleo Sol', 'LSDXOXO', 'Circuit des Yeux', 'Erika de Casier', 'Amaarae', 'Monaleo', 'Bartees Strange', 'Saweetie', 'Tirzah', 'Billie Eilish', 'Little Simz', 'The War on Drugs', 'Grouper', 'Doss', 'L-Gante', 'Oneohtrix Point Never / Elizabeth Fraser', 'Charli XCX', 'Rxk Nephew', 'Justin Bieber', 'Magdalena Bay', 'Kacey Musgraves', 'Illuminati Hotties', 'Adele', 'Playboi Carti', 'Madlib', 'Burial', 'Doja Cat', 'MIKE', 'Mdou Moctar', '

In [85]:
#Scraping all the links in the selected area
links = soup.select(".ArticlePageChunks-lgQXyi.iPDsWW a")
links

[<a class="external-link" data-event-click='{"element":"ExternalLink","outgoingURL":"https://open.spotify.com/playlist/0aAZEW9s8WDln6dN6432wc?si=3cbda337906349f1"}' data-offer-url="https://open.spotify.com/playlist/0aAZEW9s8WDln6dN6432wc?si=3cbda337906349f1" href="https://open.spotify.com/playlist/0aAZEW9s8WDln6dN6432wc?si=3cbda337906349f1" rel="nofollow noopener" target="_blank">Spotify playlist</a>,
 <a class="external-link" data-event-click='{"element":"ExternalLink","outgoingURL":"https://music.apple.com/us/playlist/pitchforks-100-best-songs-of-2021/pl.b2622a1393fb48a4b30badf8333b057a"}' data-offer-url="https://music.apple.com/us/playlist/pitchforks-100-best-songs-of-2021/pl.b2622a1393fb48a4b30badf8333b057a" href="https://music.apple.com/us/playlist/pitchforks-100-best-songs-of-2021/pl.b2622a1393fb48a4b30badf8333b057a" rel="nofollow noopener" target="_blank">Apple Music playlist</a>,
 <a href="https://pitchfork.com/topics/the-best-music-of-2021/">here</a>,
 <a href="https://www.you

In [86]:
#Keep only the links that to the Youtube Videos belonging to the songs

count_links = 0
Urls = []
for link in links:
    #Remove all special chars from both strings to compare them
    link_title = ''.join(e for e in link.get_text() if e.isalnum()) 
    chart_title = ''.join(e for e in str(titles[count_links]) if e.isalnum()) 

    #If the title of the link matches the title in the chart, the link is saved in Urls
    
    if link_title == chart_title:
        path = link['href']
        Urls.append(path)
        count_links = count_links + 1
    
print(Urls)
len(Urls)

['https://www.youtube.com/watch?v=7Azv0G85lh8', 'https://www.youtube.com/watch?v=bHp3dnAQAFc', 'https://www.youtube.com/watch?v=1sUSDQxUV7A', 'https://www.youtube.com/watch?v=AwpVNq1FeEk', 'https://www.youtube.com/watch?v=cMV4-xekoaY', 'https://www.youtube.com/watch?v=6rlkQXdji7M', 'https://www.youtube.com/watch?v=zln8SZO6yYY', 'https://www.youtube.com/watch?v=Og0Y73hAoVE', 'https://www.youtube.com/watch?v=yACWrKIKeMs', 'https://www.youtube.com/watch?v=wO7ZdCgHOYs', 'https://www.youtube.com/watch?v=ctf5J1FTEI8', 'https://www.youtube.com/watch?v=5EpnYQW8XWY', 'https://www.youtube.com/watch?v=F8NeBYucGcw', 'https://www.youtube.com/watch?v=K6kEQrog0RQ', 'https://www.youtube.com/watch?v=SwNOfTaUwrw', 'https://www.youtube.com/watch?v=62-iVogg5jA', 'https://www.youtube.com/watch?v=geclsIbWqoA', 'https://www.youtube.com/watch?v=aklK9cjLwJM', 'https://www.youtube.com/watch?v=fQ18ZZznLZE', 'https://www.youtube.com/watch?v=bd5l5NtzoWc', 'https://www.youtube.com/watch?v=6kSAfpJtULA', 'https://www

100

In [87]:
Urls[0]

'https://www.youtube.com/watch?v=7Azv0G85lh8'

Scrape Data from Youtube Links
- Views
- Likes
- Number of Comments
- Date Published


In [89]:
Views = []
Likes = []
Comments = []
DatePublished = []

'https://www.youtube.com/watch?v=lPckvzufS90'

In [90]:
#https://www.youtube.com/watch?v=lPckvzufS90
#I noticed that Urls[25], which was taken from the website, directs to a deleted video
#I have to manually assign it to the correct video of the same song
Urls[25] = 'https://www.youtube.com/watch?v=aJPRHu7i1Kk&ab_channel=BadBoyChillerCrew'

In [91]:
def youtube_authenticate():
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = "credentials.json"
    creds = None
    # the file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time
    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)
    # if there are no (valid) credentials availablle, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(client_secrets_file, SCOPES)
            creds = flow.run_local_server(port=0)
        # save the credentials for the next run
        with open("token.pickle", "wb") as token:
            pickle.dump(creds, token)

    return build(api_service_name, api_version, credentials=creds)

# authenticate to YouTube API
youtube = youtube_authenticate()

In [92]:
def get_video_id_by_url(url):
    """
    Return the Video ID from the video `url`
    """
    # split URL parts
    parsed_url = p.urlparse(url)
    # get the video ID by parsing the query of the URL
    video_id = p.parse_qs(parsed_url.query).get("v")
    if video_id:
        return video_id[0]
    else:
        raise Exception(f"Wasn't able to parse video URL: {url}")

In [93]:
def get_video_details(youtube, **kwargs):
    return youtube.videos().list(
        part="snippet,contentDetails,statistics",
        **kwargs
    ).execute()

In [104]:
def video_infos(video_response):
    items = video_response.get("items")[0]
    ## get the snippet, statistics & content details from the video response
    snippet         = items["snippet"]
    statistics      = items["statistics"]
    content_details = items["contentDetails"]
    ## get infos from the snippet
    publish_time  = snippet["publishedAt"]
    ## get stats infos
    comment_count = statistics["commentCount"]
    like_count    = statistics["likeCount"]
    view_count    = statistics["viewCount"]
    return view_count, like_count, comment_count, publish_time

In [105]:
def scrapeYT(url):
    # parse video ID from URL
    video_id = get_video_id_by_url(url)
    # make API call to get video info
    response = get_video_details(youtube, id=video_id)
    # print extracted video infos
    Views.append(int(video_infos(response)[0]))
    Likes.append(int(video_infos(response)[1]))
    Comments.append(int(video_infos(response)[2]))
    DatePublished.append(video_infos(response)[3])


In [106]:
Views.clear()
Likes.clear()
Comments.clear()
DatePublished.clear()

In [107]:
for url in Urls:
    scrapeYT(url)

In [108]:
DatePublished

['2021-08-27T04:00:00Z',
 '2021-01-13T14:00:15Z',
 '2021-11-08T14:46:16Z',
 '2021-03-24T19:00:15Z',
 '2021-08-30T04:00:07Z',
 '2021-04-16T14:01:21Z',
 '2021-06-25T04:00:32Z',
 '2021-02-21T10:08:40Z',
 '2021-04-14T13:48:29Z',
 '2021-07-09T11:55:18Z',
 '2021-02-03T10:01:51Z',
 '2021-01-25T16:00:04Z',
 '2021-09-17T04:00:12Z',
 '2021-08-13T09:07:32Z',
 '2021-06-18T16:00:32Z',
 '2021-07-07T17:00:11Z',
 '2021-02-11T05:25:44Z',
 '2021-04-30T09:00:00Z',
 '2021-05-11T14:00:23Z',
 '2021-08-29T20:14:15Z',
 '2021-03-02T19:31:06Z',
 '2021-05-05T13:36:13Z',
 '2021-06-14T20:27:17Z',
 '2021-03-24T16:00:13Z',
 '2021-07-09T00:24:12Z',
 '2022-02-18T11:06:15Z',
 '2021-06-21T18:00:12Z',
 '2021-07-21T14:00:11Z',
 '2021-04-29T23:35:34Z',
 '2021-05-19T14:00:15Z',
 '2021-04-09T12:54:38Z',
 '2021-07-16T04:00:08Z',
 '2021-05-26T14:00:58Z',
 '2020-12-20T12:34:43Z',
 '2021-02-16T18:00:12Z',
 '2021-03-25T10:25:11Z',
 '2021-01-25T18:15:29Z',
 '2021-05-06T10:00:00Z',
 '2021-03-26T04:00:14Z',
 '2021-08-19T10:09:29Z',


In [109]:
df = pd.DataFrame(range(100,0,-1),columns=['Rank'])

In [110]:
df["Song Name"] = songs
df["Artist"] = artists
df["Views"] = Views
df["Likes"] =  Likes
df["Comments"] = Comments
df["DatePublished"] = DatePublished
df["Link"] = Urls

In [111]:
df.head()

,Rank,Song Name,Artist,Views,Likes,Comments,DatePublished,Link
0,100,"“I am not a woman, I’m a god”",Halsey,18287787,368173,10502,2021-08-27T04:00:00Z,https://www.youtube.com/watch?v=7Azv0G85lh8
1,99,“Believer”,Smerz,75957,1996,86,2021-01-13T14:00:15Z,https://www.youtube.com/watch?v=bHp3dnAQAFc
2,98,“Bottle Episode”,"Mandy, Indiana",14714,375,17,2021-11-08T14:46:16Z,https://www.youtube.com/watch?v=1sUSDQxUV7A
3,97,“Last Day on Earth”,beabadoobee,821477,31694,1541,2021-03-24T19:00:15Z,https://www.youtube.com/watch?v=AwpVNq1FeEk
4,96,“Being Honest”,Kay Flock,14751121,184282,7188,2021-08-30T04:00:07Z,https://www.youtube.com/watch?v=cMV4-xekoaY


In [112]:
df.loc[df.Artist == "Kay Flock"]

,Rank,Song Name,Artist,Views,Likes,Comments,DatePublished,Link
4,96,“Being Honest”,Kay Flock,14751121,184282,7188,2021-08-30T04:00:07Z,https://www.youtube.com/watch?v=cMV4-xekoaY
